# Lab2 - Data Collection and Pre-processing-Jahnavi Pakanati-9013742

# Importing the required libraries 

In [ ]:
import pandas as pd

# Loading the  raw CSV, displaying the  first 3 rows

In [5]:
df = pd.read_csv("data/clothing_transactions.csv")
print("Columns:", df.columns.tolist())
print(df.head(20))


Columns: ['date', 'customer_id', 'product', 'price', 'quantity', 'coupon_code', 'shipping_city', 'postal_code']
          date  customer_id   product   price  quantity coupon_code  \
0   2025-05-11         1000     Scarf  116.39         1     JEANS10   
1   2025-05-03         1001   T-Shirt   90.00         4     WINTER5   
2   2025-05-02         1002     Dress   80.00         2  SPRINGSALE   
3   2025-05-11         1003    Shorts   37.67         5         NaN   
4   2025-05-06         1004  Sneakers   52.64         4    SUMMER15   
5   2025-05-08         1005    Jacket  100.86         2    SUMMER15   
6   2025-05-11         1006     Dress   40.76         4    SUMMER15   
7   2025-05-06         1007     Scarf   68.01         3     CLOTH20   
8   2025-05-07         1008    Hoodie  108.53         1         NaN   
9   2025-05-08         1009     Dress   66.90         4  SPRINGSALE   
10  2025-05-04         1010    Jacket   98.25         2     WINTER5   
11  2025-05-11         1011    Short